In [ ]:
pip install "ray[serve]"


In [1]:
import ray 

In [2]:
from ray import serve

In [3]:
import requests
from strgen import StringGenerator
ray.init()

2022-05-12 06:59:18,969	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': 'tcp://127.0.0.1:64303',
 'raylet_socket_name': 'tcp://127.0.0.1:63081',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\santi\\AppData\\Local\\Temp\\ray\\session_2022-05-12_06-59-11_161858_13472',
 'metrics_export_port': 64767,
 'node_id': '50935ffbf21b59f47b1538f452781b626506eea6de76ad71a8b8d6bf'}

In [4]:
serve.start()

 pid=17052) 2022-05-12 06:59:29,268	INFO checkpoint_path.py:16 -- Using RayInternalKVStore for controller checkpoint and recovery.
 pid=17052) 2022-05-12 06:59:29,302	INFO http_state.py:98 -- Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:TEBpDF:SERVE_PROXY_ACTOR-node:127.0.0.1-0' on node 'node:127.0.0.1-0' listening on '127.0.0.1:8000'
2022-05-12 06:59:30,096	INFO api.py:463 -- Started Serve instance in namespace '9974bdc9-a841-4dcb-a5fa-7ffda7286624'.


In [5]:
def fac(n):
    res = 1
    for i in range (1,n+1):
        res*= i
    return res

In [6]:
def fib(n):
    if n<= 1:
        return n
    else: 
        return fib(n-1)+ fib(n-2)


In [7]:
eval('2+4+fac(5)' )

126

 pid=15924) INFO:     Started server process [15924]


In [8]:
from starlette.requests import Request
from starlette.responses import Response

In [18]:
@serve.deployment

def evaluar(request):
    def fib(n):
        if n<= 1:
            return n
        else: 
            return fib(n-1)+ fib(n-2)
    def fac(n):
        res = 1
        for i in range (1,n+1):
            res*= i
        return res
    operacion = request.url.path.split("evaluar/")[-1]
    return eval(operacion)

In [10]:
evaluar.deploy()

2022-05-12 06:59:30,206	INFO api.py:242 -- Updating deployment 'evaluar'. component=serve deployment=evaluar
 pid=17052) 2022-05-12 06:59:30,301	INFO deployment_state.py:912 -- Adding 1 replicas to deployment 'evaluar'. component=serve deployment=evaluar
2022-05-12 06:59:30,994	INFO api.py:249 -- Deployment 'evaluar' is ready at `http://127.0.0.1:8000/evaluar`. component=serve deployment=evaluar


In [19]:
@ray.remote
def evaluacion_remota(entrada):
    r = requests.get('http://127.0.0.1:8000/evaluar/'+entrada)
    return r.content

In [12]:
evaluacion_remota('2+2*6')

b'14'

Relizar operaciones matematicas aleatorias

In [20]:
@ray.remote
def generar_operacion(dificultad):
    funcion=[0,0,0,0,0]
    funcion[0]= StringGenerator('[ ] |(fac)|(fib)').render()
    funcion[1]= StringGenerator('[1-9]{%s}' % dificultad).render()
    funcion[2]= StringGenerator('[+-*/%]').render()
    funcion[3]= StringGenerator('[ ]|(fac)|(fib)').render()
    funcion[4]= StringGenerator('[1-9]{%s}' % dificultad).render()
    operacions=funcion[0]+'('+funcion[1]+')'+funcion[2]+funcion[3]+'('+funcion[4]+')'
    return operacions

In [14]:
generar_operacion(1)

' fac(6)/ (4)'

In [15]:
operacion=generar_operacion(1)
resultado = evaluacion_remota(operacion)
print(operacion,resultado)

 fib(1)%fib(6) b'1'


In [16]:
evaluacion_remota(generar_operacion(1))

b'0.2833333333333333'

Apagar Ray en caso de saturarse


In [ ]:
ray.shutdown()